# TREINAMENTO PYSPARK

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# 1.0 Importando bibliotecas

In [2]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.types import *

# 2.0 Iniciando sessão

In [3]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_01')
    .getOrCreate()
)

# 3.0 Importando dataset

In [4]:
df = spark.read.csv('wc2018-players.csv', header=True, inferSchema=True)

In [5]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



# 4.0 Verificando tipos das colunas

In [6]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



# 5.0 Renomeando coluna

In [7]:
df = df.withColumnRenamed('Pos.', 'Posicao').withColumnRenamed('Team', 'Selecao').withColumnRenamed('FIFA Popular Name', 'Nome_FIFA')\
.withColumnRenamed('Birth Date', 'Nascimento').withColumnRenamed('Shirt Name', 'Nome_Camiseta')\
.withColumnRenamed('Club', 'Time').withColumnRenamed('Height', 'Altura')\
.withColumnRenamed('Weight', 'Peso')

# 6.0 Verificando dados nulos

In [8]:
# se o df não for tão grande

df.toPandas().isna().sum()

Selecao          0
#                0
Posicao          0
Nome_FIFA        0
Nascimento       0
Nome_Camiseta    0
Time             0
Altura           0
Peso             0
dtype: int64

In [9]:
# se o df for grande

for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Selecao 0
# 0
Posicao 0
Nome_FIFA 0
Nascimento 0
Nome_Camiseta 0
Time 0
Altura 0
Peso 0


# 7.0 Selecionar colunas

In [10]:
df.select('Selecao','Nome_FIFA').show(5)

+---------+------------------+
|  Selecao|         Nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [11]:
df.select(col('Selecao'), col('Altura')).show(5)

+---------+------+
|  Selecao|Altura|
+---------+------+
|Argentina|   169|
|Argentina|   169|
|Argentina|   167|
|Argentina|   167|
|Argentina|   170|
+---------+------+
only showing top 5 rows



In [12]:
df.select(df['Selecao']).show(5)

+---------+
|  Selecao|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



Selecionando com ALIAS

In [13]:
df.select(col('Selecao').alias('Time')).show(5)

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



# 8.0 Organizando select

In [14]:
df.select('Nome_fifa','Peso','Altura').show(5)

+------------------+----+------+
|         Nome_fifa|Peso|Altura|
+------------------+----+------+
|TAGLIAFICO Nicolas|  65|   169|
|    PAVON Cristian|  65|   169|
|    LANZINI Manuel|  66|   167|
|    SALVIO Eduardo|  69|   167|
|      MESSI Lionel|  72|   170|
+------------------+----+------+
only showing top 5 rows



# 9.0 Filtrar as linhas

In [16]:
df.filter('Selecao="Brazil"').show(5)

+-------+---+-------+-----------+----------+-------------+--------------------+------+----+
|Selecao|  #|Posicao|  Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+-------+---+-------+-----------+----------+-------------+--------------------+------+----+
| Brazil| 18|     MF|       FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil| 21|     FW|     TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil| 17|     MF|FERNANDINHO|04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil| 22|     DF|     FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil| 10|     FW|     NEYMAR|05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
+-------+---+-------+-----------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [18]:
df.filter(col('Nome_Camiseta')=="FRED").show()

+-------+---+-------+---------+----------+-------------+--------------------+------+----+
|Selecao|  #|Posicao|Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+-------+---+-------+---------+----------+-------------+--------------------+------+----+
| Brazil| 18|     MF|     FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
+-------+---+-------+---------+----------+-------------+--------------------+------+----+



#### Filtrar com 2 condições (AND/&)

In [20]:
df.filter((col('Selecao')=='Argentina') & (col('Altura')>180)).show()

+---------+---+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|  #|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|  4|     DF|  ANSALDI Cristian|20.09.1986|      ANSALDI|     Torino FC (ITA)|   181|  73|
|Argentina|  9|     FW|   HIGUAIN Gonzalo|10.12.1987|      HIGUAÍN|   Juventus FC (ITA)|   184|  75|
|Argentina| 23|     GK|CABALLERO Wilfredo|28.09.1981|    CABALLERO|    Chelsea FC (ENG)|   186|  80|
|Argentina|  2|     DF|   MERCADO Gabriel|18.03.1987|      MERCADO|    Sevilla FC (ESP)|   181|  81|
|Argentina| 17|     DF|  OTAMENDI Nicolas|12.02.1988|     OTAMENDI|Manchester City F...|   181|  81|
|Argentina| 16|     DF|       ROJO Marcos|20.03.1990|         ROJO|Manchester United...|   189|  82|
|Argentina|  6|     DF|    FAZIO Federico|17.03.1987|        FAZIO|       AS Roma (ITA)|   

#### OR / |

In [23]:
df.filter((col('Selecao')=='Argentina') | (col('Altura')>180)).show(5)

+---------+---+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|  #|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|  3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina| 22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina| 15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina| 18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina| 10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



#### AND e OR

In [26]:
df.filter((col('Selecao')=='Brazil') & (col('Posicao')=='DF') | (col('Nome_FIFA')=='MESSI Lionel')).show(5)

+---------+---+-------+------------+----------+-------------+--------------------+------+----+
|  Selecao|  #|Posicao|   Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+---------+---+-------+------------+----------+-------------+--------------------+------+----+
|Argentina| 10|     FW|MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
|   Brazil| 22|     DF|      FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
|   Brazil|  6|     DF| FILIPE LUIS|09.08.1985|  FILIPE LUIS|Atletico Madrid (...|   182|  73|
|   Brazil| 13|     DF|  MARQUINHOS|14.05.1994|   MARQUINHOS|Paris Saint-Germa...|   183|  75|
|   Brazil|  3|     DF|     MIRANDA|07.09.1984|      MIRANDA|FC Internazionale...|   186|  78|
+---------+---+-------+------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



# 10. Criando novas colunas

In [27]:
df.withColumn('WorldCup', lit(2018)).show(5)

+---------+---+-------+------------------+----------+-------------+--------------------+------+----+--------+
|  Selecao|  #|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|WorldCup|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+--------+
|Argentina|  3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|
|Argentina| 22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|    2018|
|Argentina| 15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|    2018|
|Argentina| 18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|    2018|
|Argentina| 10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|    2018|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+--------+
only showi

In [28]:
df.withColumn('IMC', lit(col('Altura')/col('Peso'))).show(5)

+---------+---+-------+------------------+----------+-------------+--------------------+------+----+------------------+
|  Selecao|  #|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|               IMC|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+------------------+
|Argentina|  3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|               2.6|
|Argentina| 22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|               2.6|
|Argentina| 15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|2.5303030303030303|
|Argentina| 18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69| 2.420289855072464|
|Argentina| 10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72| 2.361111111111111|
+---------+---+-------+-----------------

#### Coluna condicional

In [29]:
df.withColumn('Sub', substring('Selecao', 1, 3)).show(5)

+---------+---+-------+------------------+----------+-------------+--------------------+------+----+---+
|  Selecao|  #|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|Sub|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+---+
|Argentina|  3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|Arg|
|Argentina| 22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|Arg|
|Argentina| 15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|Arg|
|Argentina| 18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|Arg|
|Argentina| 10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|Arg|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+---+
only showing top 5 rows



#### Concatenar

In [30]:
df.withColumn('Concat', concat('Selecao', 'Nome_camiseta')).show(5)

+---------+---+-------+------------------+----------+-------------+--------------------+------+----+-------------------+
|  Selecao|  #|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|             Concat|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+-------------------+
|Argentina|  3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|ArgentinaTAGLIAFICO|
|Argentina| 22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|     ArgentinaPAVÓN|
|Argentina| 15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|   ArgentinaLANZINI|
|Argentina| 18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|    ArgentinaSALVIO|
|Argentina| 10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|     ArgentinaMESSI|
+---------+---+-------+---------

In [32]:
df.withColumn('Separador', concat_ws(' - ', 'Selecao', 'Nascimento')).show(5)

+---------+---+-------+------------------+----------+-------------+--------------------+------+----+--------------------+
|  Selecao|  #|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|           Separador|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+--------------------+
|Argentina|  3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|Argentina - 31.08...|
|Argentina| 22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|Argentina - 21.01...|
|Argentina| 15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|Argentina - 15.02...|
|Argentina| 18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|Argentina - 13.07...|
|Argentina| 10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|Argentina - 24.06...|
+---------+---+-------+-

# 11. Alterar tipagem dos dados

In [33]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome_Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)



In [36]:
df = df.withColumn('Ano',substring('Nascimento', -4, 4))

In [37]:
df.show(5)

+---------+---+-------+------------------+----------+-------------+--------------------+------+----+----+
|  Selecao|  #|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso| Ano|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+----+
|Argentina|  3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|
|Argentina| 22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|
|Argentina| 15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|
|Argentina| 18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|
|Argentina| 10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+----+
only showing top 5 rows



In [40]:
df = df.withColumn('Ano', col('Ano').cast(IntegerType()))

In [41]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome_Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)



#### Coluna de Nascimento

In [42]:
df.withColumn('Nascimento2', regexp_replace('Nascimento', '.', '-')).show()

+---------+---+-------+------------------+----------+-------------+--------------------+------+----+----+-----------+
|  Selecao|  #|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso| Ano|Nascimento2|
+---------+---+-------+------------------+----------+-------------+--------------------+------+----+----+-----------+
|Argentina|  3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992| ----------|
|Argentina| 22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996| ----------|
|Argentina| 15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993| ----------|
|Argentina| 18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990| ----------|
|Argentina| 10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987| ----------|
|Argentina|  4|     DF|  ANSALDI Cristian|20.09.1986|   